In [14]:
"""
Simple tester for the deep3d
"""

import tensorflow as tf
import vgg19_trainable as vgg19
import utils
import numpy as np
import os
import random
import os.path
import h5py
import matplotlib as plt

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
#dataset will be 224 x 224 crops out of the inria dataset, with random labels from 1 to 10

inria_file = '/a/data/deep3d/inria_data.h5'
h5f = h5py.File(inria_file,'r')

X_train = np.zeros((400,224,224,3))
Y_train = np.random.randint(10,size=400)
X_val = np.zeros((50,224,224,3))
Y_val = np.random.randint(10,size=50)
X_train[:,22:202,:,:] = h5f['X_0'][:400,:,48:272,:]
X_val[:,22:202,:,:] = h5f['X_0'][400:450,:,48:272,:]

h5f.close()

In [17]:
print Y_train.shape

(400,)


In [18]:
num_batches = 10
batchsize = 40
device1 = '/gpu:0'
#device1 = '/cpu:0'
with tf.device(device1):
    sess = tf.Session()

    images = tf.placeholder(tf.float32, [batchsize, 224, 224, 3])
    true_out = tf.placeholder(tf.float32, batchsize)
    
    train_mode = tf.placeholder(tf.bool)

    net = vgg19.Vgg19('./vgg19.npy')
    net.build(images, train_mode)

    # print number of variables used: 143667240 variables, i.e. ideal size = 548MB
    print 'var count:'
    print(net.get_var_count())

    sess.run(tf.global_variables_initializer())

    # simple 1-step training
    cost = tf.reduce_sum((net.prob - true_out) ** 2)
    train = tf.train.GradientDescentOptimizer(0.002).minimize(cost)
    for i in xrange(num_batches):
        image_mask = np.random.choice(X_train.shape[0],batchsize)
        images_in = X_train[image_mask,:,:,:]
        labels_in = Y_train[image_mask,:,:,:]
        sess.run(train, feed_dict={images: images_in, true_out: labels_in, train_mode: True})
        print 'cost:'
        print cost.eval(session=sess)
    # test classification again, should have a higher probability about tiger
    #prob = sess.run(vgg.prob, feed_dict={images: batch1, train_mode: False})
    #utils.print_prob(prob[0], './synset.txt')

    # test save
    #vgg.save_npy(sess, './test-save.npy')

[224, 224, 1]
var count:
143667240


InternalError: Dst tensor is not initialized.
	 [[Node: fc6_1/fc6_weights/initial_value = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [25088,4096] values: [0.00074060471 -9.0235051e-05 0.00041919478]...>, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op u'fc6_1/fc6_weights/initial_value', defined at:
  File "/usr/lib64/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib64/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/a/h/jhohra01/Envs/deep-venv/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/a/h/jhohra01/Envs/deep-venv/lib/python2.7/site-packages/traitlets/config/application.py", line 592, in launch_instance
    app.start()
  File "/a/h/jhohra01/Envs/deep-venv/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 405, in start
    ioloop.IOLoop.instance().start()
  File "/a/h/jhohra01/Envs/deep-venv